In [ ]:
## import pymc3 as pm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from numpy.random import seed
from sklearn.preprocessing import MinMaxScaler, StandardScaler,RobustScaler
RANDOM_SEED = 2018
seed(RANDOM_SEED)
tf.set_random_seed(RANDOM_SEED)
data_path = '/power_consumption/data/'

In [ ]:
consumption_full = pd.read_csv(data_path + 'consumption_train.csv', parse_dates=['timestamp'])
consumption_full = consumption_full.drop('Unnamed: 0', axis=1)
t = consumption_full.head()


In [ ]:
scaler = {}
c_f = {}
for ser_id, ser_df in consumption_full.groupby('series_id'):
    consumption = ser_df.consumption.reshape(-1,1)
#     scaler[ser_id] = MinMaxScaler() # range(0,1)
    scaler[ser_id] = StandardScaler()
    scaler[ser_id].fit(consumption)
    t = scaler[ser_id].transform(consumption)
    ser_df['consumption_norm'] = t
    if type(c_f) == pd.DataFrame:
        c_f = c_f.append(ser_df)
    else:
        c_f = ser_df
consumption_full = c_f

In [ ]:
consumption_full

In [ ]:
consumption_meta = pd.read_csv(data_path + 'meta.csv')
# consumption_meta.surface.unique()
consumption_meta.head()

In [ ]:
sizes = {
    'xx-small': 0,
    'x-small': 0.15,
    'small': 0.3,
    'medium': 0.5,
    'large': 0.65,
    'x-large': 0.8,
    'xx-large': 1.0
}
on_off = {
    0: 'monday_is_day_off',
    1: 'tuesday_is_day_off',
    2: 'wednesday_is_day_off',
    3: 'thursday_is_day_off',
    4: 'friday_is_day_off',
    5: 'saturday_is_day_off',
    6: 'sunday_is_day_off'
}
consumption_meta['area'] = consumption_meta.surface.map(lambda x: sizes[x])
# consumption_meta.apply(lambda row: row.series_id, axis=1)

In [ ]:
# pd.to_datetime(consumption_full.timestamp).map(lambda x: x.dayofweek)
consumption_full['day_of_week'] = consumption_full.timestamp.map(lambda x: (x.dayofweek/6.0))
consumption_full['hour'] = consumption_full.timestamp.map(lambda x: (x.hour/23.0))
# consumption_full['area'] = consumption_full.series_id.map(lambda x: (
#     consumption_meta[consumption_meta.series_id == x].area.values[0]
# ))
def is_on(row):
    try:
        dow = row.timestamp.dayofweek
        off = consumption_meta[consumption_meta.series_id == row.series_id][on_off[dow]].values[0]
    except Exception as e:
#         print(consumption_meta[consumption_meta.series_id == row.series_id])
        print(not consumption_meta[consumption_meta.series_id == row.series_id][on_off[dow]].values[0])
        raise e
    return not off

consumption_full['is_on'] = consumption_full.apply(is_on, axis=1)

consumption_full.head(20)

In [ ]:
consumption_full['n_is_on'] = consumption_full.is_on +0

In [ ]:
len(consumption_full.day_of_week.unique())

In [ ]:
# choose subset of series for training
frac_series_to_use = 0.7

rng = np.random.RandomState(seed=RANDOM_SEED)
series_ids = c_f.series_id.unique()
training_mask = rng.binomial(1,
                           frac_series_to_use,
                           size=series_ids.shape).astype(bool)
testing_mask = ~training_mask
training_series = series_ids[training_mask]
testing_series = series_ids[testing_mask]

# reduce training data to series subset
consumption_train = consumption_full[consumption_full.series_id.isin(training_series)]
consumption_test = consumption_full[consumption_full.series_id.isin(testing_series)]


In [ ]:
n_is_on_idx = 0
hour_idx = 1
day_of_week_idx = 2
consumption_norm_idx = 3
def create_lagged_features(df, lag=1):
    # Takes in DF
    if not type(df) == pd.DataFrame:
#         df = pd.DataFrame(df, columns=['consumption_norm'])
        raise Exception('NOT A DATAFRAME')
    def _rename_lag(ser, j):
        ser.name = ser.name + '_{j}'.format(j=j)
        return ser
    if len(df) == 24:
        last_row = df[-1:]
        df = df.append(df[-1:], ignore_index=True)
    df = df[['consumption_norm', 'day_of_week', 'hour', 'n_is_on']]
    for i in range(1, lag+1):
        df = df.join(df[['consumption_norm', 'day_of_week', 'hour', 'n_is_on']].shift(i), rsuffix=i)#.pipe(_rename_lag, i))
    df.dropna(inplace=True)
    return df

create_lagged_features(consumption_full[consumption_full.series_id==100003], lag=24).head()
#     [cold_start_test.series_id==100090].consumption, lag=24
# ).head()[-1:]

In [ ]:
tf.reset_default_graph()
# DEFINE THE TF MODEL
# * FCNN with 2 hidden layers
# Define input/output vars
# Include rolling average as input(weekly, monthly)
input_num_units = 24
output_num_units = 4
h1_units = 48
h2_units = 28
metad_size = 1
num_neurons = 64
lstm_sizes = [num_neurons, num_neurons/2, num_neurons/2, num_neurons/2, num_neurons]#np.ceil(0.5 * num_neurons), np.ceil(0.5 * num_neurons)]
# dropout_keep_prob = 0.9
dropout_keep_prob=1

x = tf.placeholder(tf.float32, [None, input_num_units, output_num_units])
y = tf.placeholder(tf.float32, [None, output_num_units])
# observed_mean = tf.placeholder(tf.float32, [None, 1])
# Define hyperparams

epochs = 1
batch_size = 1
learning_rate = 0.0001
weights = {
#     'h1': tf.Variable(tf.random_normal([int(input_num_units * lstm_sizes[-1] + metad_size), h1_units])),
    'h1': tf.Variable(tf.random_normal([int(lstm_sizes[-1]), h1_units])),
#     'h2': tf.Variable(tf.random_normal([h1_units, h2_units])),
    'output': tf.Variable(tf.random_normal([h1_units, output_num_units]))
}

biases = {
    'h1': tf.Variable(tf.random_normal([h1_units])),
#     'h2': tf.Variable(tf.random_normal([h2_units])),
    'output': tf.Variable(tf.random_normal([output_num_units]))
}

lstms = [tf.nn.rnn_cell.BasicLSTMCell(num_units=size, activation=tf.nn.elu)
         for size in lstm_sizes]
drops = [tf.nn.rnn_cell.DropoutWrapper(lstm, output_keep_prob=dropout_keep_prob) for lstm in lstms]
cell = tf.nn.rnn_cell.MultiRNNCell(drops)

init_state = cell.zero_state(batch_size, tf.float32)
lstm_outputs, final_state = tf.nn.dynamic_rnn(cell, x, initial_state=init_state)
# lstm_outputs: (?, 24, 30) or (len(X), num_inputs, num_neurons in last lstm layer)

# Select only last output from LSTM to feed to FC 
# trunc_lstm_out = tf.reshape(lstm_outputs, [-1, input_num_units * lstm_sizes[-1]] )
trunc_lstm_out = lstm_outputs[:,-1]
# trunc_lstm_out: (?, 30)

# * Combine stacked_lstm_outputs with series metadata to pass to FC layersii
# aug_stacked = tf.concat([trunc_lstm_out, observed_mean], 1)
# h1_layer = tf.add(tf.matmul(aug_stacked, weights['h1']), biases['h1'])
h1_layer = tf.add(tf.matmul(trunc_lstm_out, weights['h1']), biases['h1'])
h1_layer = tf.nn.relu(h1_layer)

# h2_layer = tf.add(tf.matmul(h1_layer, weights['h2']), biases['h2'])
# h2_layer = tf.nn.relu(h2_layer)

output_layer = tf.add(tf.matmul(h1_layer, weights['output']), biases['output'])

cost = tf.reduce_mean(tf.abs(output_layer - y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)


In [ ]:
c_t = []
test_len = 1000 # Number of series to test
lag = 24
epochs = 1
checkpoint = './pc_v5_n_is_on.ckpt'
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver = tf.train.Saver()
#     saver.restore(sess, checkpoint)
    for i in range(epochs):
        print("Epoch: " + str(i))
        ctr = 0
        for ser_id, ser_df in consumption_train.groupby('series_id'):
            print('#' + str(ctr) + ': ' + str(ser_id))
            ctr += 1
            if ctr == test_len:
                print('REACHED TEST LENGTH, STOPPING')
                break
            state = sess.run(init_state) # Re-initializes empty state
            train_lagged = create_lagged_features(ser_df, lag=lag).values
            train_lagged = np.flip(train_lagged, axis=1)
            train_scaled_in = train_lagged[:,:-output_num_units].reshape(-1, lag, output_num_units)
            train_scaled_out = train_lagged[:,-output_num_units:].reshape(-1, output_num_units)
            for j in range(int(np.floor(train_scaled_in.shape[0]/batch_size))):
#                 state, out = sess.run([final_state, output_layer], feed_dict={
                state, out = sess.run([final_state, optimizer], feed_dict={
                    x: train_scaled_in[batch_size*j:batch_size*j+batch_size],
                    y: train_scaled_out[batch_size*j:batch_size*j+batch_size],
                    init_state: state
                })

                if j%100 == 0:
                    h, c = sess.run([output_layer, cost], feed_dict={
                        x: train_scaled_in[batch_size*j:batch_size*j+batch_size],
                        y: train_scaled_out[batch_size*j:batch_size*j+batch_size],
                        init_state: state
                    })
                    print("Cost @ " + str(j) + ': ' + str(c))
                    print('Prediction: ' + str(h))
                    print('Actual: ' + str(train_scaled_out[batch_size*j:batch_size*j+batch_size]))
#                           str(h[0][0]
#                               - train_scaled_out[batch_size*j:batch_size*j+batch_size][0][0]))
            saver.save(sess, checkpoint)
#         c_t.append(sess.run(cost, feed_dict={
#             x: train_scaled_in,
#             y: train_scaled_out
#         }))
#         print('Epoch :',i,'Cost :',c_t[i])
    
        

In [ ]:
cold_start_len = 24 * 2
consumption_test['prediction'] = np.nan
consumption_test['error'] = np.nan

# Returns error score for series_id
def run_test(series_id, df_return=False):
    with tf.Session() as local_sess:
        saver_i = tf.train.Saver()
        cold_start_len = 24 * 2

        saver_i.restore(local_sess, checkpoint) # Start each series from scratch to avoid test-order effects
        test_cold_start = (consumption_test
                           [consumption_test.series_id == series_id].dropna(subset=['consumption_norm'])
                           [:cold_start_len])
        test_pred = consumption_test[consumption_test.series_id == series_id][cold_start_len:]
        cold_input = np.flip(create_lagged_features(
            test_cold_start,
            lag=24
        ).values, axis=1)
        state = local_sess.run(init_state) # Zero-state
        for cold_in in cold_input:
            c_in = cold_in[:-1*output_num_units].reshape(-1, 24, output_num_units)
            cold_out = [cold_in[-1*output_num_units:]]
            h, state = local_sess.run([optimizer, final_state], feed_dict={
                x: c_in,
                y: cold_out,
                init_state: state
            })
        X = c_in
        X[0] = np.roll(X[0], -1, axis=0)
        X[0][-1] = cold_out[0]
        last_state = state
        count = 0
        for idx, row in test_pred.iterrows():
            last_last_state = last_state
            last_state = state
            if count > 0:
    #             print(count, test_pred, h);raise
                X[0][-1][day_of_week_idx] = row.day_of_week
                X[0][-1][n_is_on_idx] = row.n_is_on
                X[0][-1][hour_idx] = row.hour
            count += 1

            h, state = local_sess.run([output_layer, final_state], feed_dict={
                x: X,
                init_state: state
            })
            prediction = scaler[series_id].inverse_transform(h[:, -1].reshape(-1, 1))
#             if prediction < 0:
#                 h[0][consumption_norm_idx] = -1
#             h[0][consumption_norm_idx] = -0.5
            prediction = scaler[series_id].inverse_transform(h[:, -1].reshape(-1, 1))
            test_pred.at[idx,'prediction'] = prediction
            test_pred.at[idx, 'error'] = abs(row.consumption - prediction)
            # Push all values to the left
            X = np.array([np.roll(X[0], -1, axis=0)])
            # Replace the (formerly last) value with latest prediction.
            X[0][-1] = h
        err = test_pred.error.mean()/test_pred.consumption.mean()
        if df_return:
            return (err, test_pred)
        else:
            return err
    

In [ ]:
errors = []
for series_id in consumption_test[:int(np.ceil(len(consumption_test)))].series_id.unique():
    errors.append((series_id, run_test(series_id, df_return=False)))
errors

In [ ]:
err_df = pd.DataFrame(data=errors, columns=['series_id', 'error'])

In [ ]:
# err_df.sort_values('error', ascending=False).head()
err_df.sort_values('error')

In [ ]:
for ser_id in err_df.sort_values('error').series_id[-5:]:
    print(ser_id)
    res = run_test(ser_id, df_return=True)
    print(ser_id, res[0])
    res[1].plot(
        x='timestamp', y=['consumption', 'prediction'],
        figsize=(20,5)
    )

In [ ]:
ct = consumption_test
ct[ct.prediction > 0]

In [ ]:
cold_start_test = pd.read_csv(data_path + 'cold_start_test.csv',
                              parse_dates=['timestamp'], 
                              index_col=0)
cold_start_test['hour'] = cold_start_test.timestamp.map(lambda x: (x.hour/23.0))
cold_start_test['day_of_week'] = cold_start_test.timestamp.map(lambda x: (x.dayofweek/6.0))
cold_start_test['is_on'] = cold_start_test.apply(is_on, axis=1)
cold_scaler = {}
cold_df = {}
for ser_id, ser_df in cold_start_test.groupby('series_id'):
    consumption = ser_df.consumption.reshape(-1,1)
#     cold_scaler[ser_id] = MinMaxScaler() # range(0,1)
    cold_scaler[ser_id] = StandardScaler()
    cold_scaler[ser_id].fit(consumption)
    t = cold_scaler[ser_id].transform(consumption)
    ser_df['consumption_norm'] = t
    if type(cold_df) == pd.DataFrame:
        cold_df = cold_df.append(ser_df)
    else:
        cold_df = ser_df

In [ ]:
# cold_start_test['n_is_on'] = cold_start_test.is_on +0.0
cold_df['n_is_on'] = cold_df.is_on + 0.0

In [ ]:
cold_start_test.n_is_on.unique()

In [ ]:
submission_format = pd.read_csv(data_path + 'submission_format.csv',
                               index_col = 'pred_id',
                               parse_dates=['timestamp'])
submission_format.head()

In [ ]:
my_submission = submission_format.copy()

In [ ]:
s_i.close()
s_i = tf.InteractiveSession()
saver_i = tf.train.Saver()
# saver_i.restore(s_i, './pc_v5.ckpt')

In [ ]:
on_off = {
    0: 'monday_is_day_off',
    1: 'tuesday_is_day_off',
    2: 'wednesday_is_day_off',
    3: 'thursday_is_day_off',
    4: 'friday_is_day_off',
    5: 'saturday_is_day_off',
    6: 'sunday_is_day_off'
}
def next_vals(h, ser_id):
    this_hour = h[hour_idx]
    this_day = h[day_of_week_idx]
    this_is_on = h[n_is_on_idx]
    if round(this_hour * 23) == 23.0:
        next_hour = 0.0
        if round(this_day * 6) == 6.0:
            next_day = 0
        else:
            next_day = this_day + 1.0/6.0
    else:
        next_hour = this_hour + 1.0/23.0
        next_day = this_day
    next_is_on = (not consumption_meta[consumption_meta.series_id == ser_id][on_off[int(round(next_day * 6))]].values[0]) + 0.0
    return (next_hour, next_day, next_is_on)

In [ ]:
lag=24
pred_window_to_num_preds = {'hourly': 24, 'daily': 7, 'weekly': 2}
pred_window_to_num_pred_hours = {'hourly': 24, 'daily': 7 * 24, 'weekly': 2 * 7 * 24}

num_test_series = my_submission.series_id.nunique()
cnt = 0
for ser_id, pred_df in my_submission.groupby('series_id'):
    cnt += 1
    if cnt == 5:
        break
    # Restore training 
    saver_i.restore(s_i, checkpoint)
    learning_rate = 0.001
    print('Starting ' + str(ser_id))
#     cold_df[cold_df.series_id == ser_id].consumption_norm.plot()

    state = s_i.run(init_state)
    # get info about this series' prediction window
    pred_window = pred_df.prediction_window.unique()[0]
    num_preds = pred_window_to_num_preds[pred_window]
    num_pred_hours = pred_window_to_num_pred_hours[pred_window]
    # prepare cold start data
    series_data = cold_df[cold_df.series_id == ser_id]
    cold_lagged = create_lagged_features(series_data, lag=lag).values
    cold_lagged = np.flip(cold_lagged, axis=1)
    cold_lagged_in = cold_lagged[:,:-1*output_num_units]
    cold_lagged_out = cold_lagged[:,-1*output_num_units:].reshape(-1, output_num_units)
#     print("IN:", cold_lagged_in)
#     print("OUT:", cold_lagged_out);raise    
    # fine tune our lstm model to this site using cold start data
    for i, one_in in enumerate(cold_lagged_in):
        x_in = one_in.reshape(-1, 24, output_num_units)
#         print("IN", x_in.shape)
#         print("OUT", cold_lagged_out[i:i+1]);raise
        state, o = s_i.run([final_state, optimizer], feed_dict={
            x: x_in,
            y: cold_lagged_out[i:i+1],
            init_state: state
        })
    
    # Seed X for prediction with last input of warm-up
    running_vals = np.roll(cold_lagged_in[-1].reshape(1, 24, output_num_units), -1, axis=1)
    running_vals[0][-1] = cold_lagged_out[-1]
    preds = np.array([])
    for j in range(num_pred_hours):
        pred, state = s_i.run([output_layer, final_state], feed_dict={
            x: running_vals,
            init_state: state
        })
#         print(running_vals[0][-2])
#         print(running_vals[0][-2])
#         print(next_vals(running_vals[0][-1], ser_id));raise
        next_hour, next_dow, next_n_is_on = next_vals(running_vals[0][-1], ser_id)
        running_vals = np.roll(running_vals, -1, axis=1)
        running_vals[0][-1] = pred[0]
        running_vals[0][-1][n_is_on_idx] = next_n_is_on
        running_vals[0][-1][hour_idx] = next_hour
        running_vals[0][-1][day_of_week_idx] = next_dow
        t_pred = cold_scaler[ser_id].inverse_transform([pred[0,-1]])

        if t_pred < 0 or t_pred > 1e8:
            # If prediction is out of bounds (0,1e8), use the average
            running_vals[0][-1][-1] = 0
            t_pred = cold_scaler[ser_id].inverse_transform([0.0])

        preds = np.append(preds, t_pred)
    # reduce by taking sum over each sub window in pred window
    reduced_preds = [pred.sum() for pred in np.split(preds, num_preds)]
#     ======DEBUG PLOT======
    series_data.consumption.append(pd.Series(preds)).reset_index().plot(figsize=(30,10))
#     ======++++++++++======
    # store result in submission DataFrame
    ser_id_mask = my_submission.series_id == ser_id
    my_submission.loc[ser_id_mask, 'consumption'] = reduced_preds

In [ ]:
sub.to_csv("my_submmission.csv", index_label='pred_id')

In [ ]:
blah = sub.fillna(value=0).groupby('series_id').max()
len(blah[blah.temperature==0])/len(blah)

In [ ]:
# sub2 = sub.fillna(value=0)[(sub.consumption.isnull())]
sub2 = sub.fillna(value=sub[sub.prediction_window == 'weekly'].consumption.mean())

In [ ]:
sub2.to_csv('sub2_filled_nas.csv', index_label='pred_id')